모듈 load

In [16]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics


train set load

In [17]:
trainSet = pd.read_csv("./fashion-mnist_train.csv")
# load train data and copy them
# df_train = trainSet.copy()


In [19]:
# separating data and label
X_train = trainSet.drop(['label'],axis = 1)
X_label = trainSet['label']


# do nomalization
X_train = X_train.astype('float32')
X_train /= 255.0

#X_label
X_train.head()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.019608,0.0,0.0,...,0.000000,0.0,0.0,0.117647,0.168627,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.003922,0.007843,0.0,0.0,0.000000,0.0,0.0,...,0.011765,0.0,0.0,0.000000,0.000000,0.003922,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0


train set divide -> train, val

In [20]:
#split train&test
seed = 99
np.random.seed(seed)
X_train, X_val,label_train,label_val = train_test_split(X_train,X_label,test_size =0.1, random_state= seed)
# do standard scaling
print(type(X_train))
X_train=StandardScaler().fit_transform(X_train)
X_val=StandardScaler().fit_transform(X_val)
print(type(X_train))


<class 'pandas.core.frame.DataFrame'>
<class 'numpy.ndarray'>


In [21]:
# PCA -> dimenstion Reducing
pca = PCA(n_components=187, random_state=42)
# n_components = 100 ~ 200
# n_components = 187 --> by siyun.ipynb
X_train_pca =pca.fit_transform(X_train)
X_val_pca = pca.transform(X_val)
X_train_PCA1 = pd.DataFrame(X_train_pca)
X_val_PCA1 = pd.DataFrame(X_val_pca)
X_train_pca.shape

# plt.plot(np.cumsum(pca.explained_variance_ratio_))
# plt.xlabel('number of components')
# plt.ylabel('cumulative explained variance')

(54000, 187)

In [22]:
# svm model
start = time.time()

# Cost -->  13 / kernel --> gaussian /  gamma --> 1/# features

# svc = SVC(C=1,kernel='rbf',gamma="auto",probability = True) , n_component = 100
# Train Accuracy score: 0.9651296296296297
# Test Accuracy score: 0.874
# time = 22m


# svc = SVC(C=1,kernel='rbf',gamma="scale",probability = True)
# Train Accuracy score: 0.9010185185185186
# Test Accuracy score: 0.8891666666666667
# time = 5m

# svc = SVC(C=0.9,kernel='rbf',gamma="auto",probability = True)
# Train Accuracy score: 0.9613518518518519
# Test Accuracy score: 0.8723333333333333
# time = 17m

svc = SVC(C=1,kernel='rbf',gamma="scale",probability = True) # n_ component = 200
# Train Accuracy score: 0.9113888888888889
# Test Accuracy score: 0.8945
# time = 8m
#svc = SVC(C=1,kernel='rbf',gamma="auto",probability = True)
svc.fit(X_train_PCA1,label_train)

end = time.time()
svm_time = end-start
print("SVM Time: {:0.2f} minute".format(svm_time/60.0))

SVM Time: 7.87 minute


In [7]:
# # 혼성모델
# start1 = time.time()

# random_forest = RandomForestClassifier(criterion='entropy', max_depth=70, n_estimators=100)
# random_forest.fit(X_train_PCA1, Y_train)

# end1 = time.time()
# forest_time = end1-start1
# print("Random Forest Time: {:0.2f} minute".format(forest_time/60.0))

In [12]:
# 정확도 평가
train_pred = svc.predict(X_train_PCA1)
val_pred = svc.predict(X_val_PCA1)
# X_test_pca
svc_train = metrics.accuracy_score(label_train,train_pred)
svc_accuracy = metrics.accuracy_score(label_val, val_pred)

print("Train Accuracy score: {}".format(svc_train))
print("Validation Accuracy score: {}".format(svc_accuracy))
print(metrics.classification_report(label_val, val_pred))

# y_train_forest = random_forest.predict(X_train_PCA1)
# y_pred_forest = random_forest.predict(X_test_pca)
# random_forest_train = metrics.accuracy_score(Y_train,y_train_forest)
# random_forest_accuracy = metrics.accuracy_score(Y_val, y_pred_forest)

# print("Train Accuracy score: {}".format(random_forest_train))
# print("Test Accuracy score: {}".format(random_forest_accuracy))
# print(metrics.classification_report(Y_val, y_pred_forest))

Train Accuracy score: 0.9105740740740741
Validation Accuracy score: 0.895
              precision    recall  f1-score   support

           0       0.82      0.87      0.84       574
           1       0.99      0.97      0.98       596
           2       0.82      0.83      0.82       573
           3       0.90      0.92      0.91       624
           4       0.85      0.85      0.85       635
           5       0.97      0.94      0.96       632
           6       0.76      0.68      0.71       600
           7       0.94      0.96      0.95       604
           8       0.94      0.98      0.96       598
           9       0.96      0.96      0.96       564

    accuracy                           0.90      6000
   macro avg       0.89      0.89      0.89      6000
weighted avg       0.89      0.90      0.89      6000



In [23]:
# test set load
import os
from PIL import Image

data_dir = "./data/data"
file_names = [f for f in os.listdir(data_dir) if os.path.isfile(os.path.join(data_dir, f))]
Test_set = []
for file in file_names:
    if file.endswith('png'):
        file_path = os.path.join(data_dir,file)
        img = Image.open(file_path)
        if img.mode == 'RGB':
            img = img.convert('L')
        img_arr = list(img.getdata())
        Test_set.append(img_arr)


columns = [f'pixel_{i}' for i in range(28*28)]

test_X = pd.DataFrame(Test_set, columns=columns)
test_X = test_X.astype('float32')
test_X /= 255.0

test_df = StandardScaler().fit_transform(test_X)




In [24]:
test_pca =pca.transform(test_df)
test_PCA = pd.DataFrame(test_pca)
test_pred = svc.predict(test_PCA)

clip = test_pred[:10000]
with open("./mAP/testResult.txt", "w") as f:
    for f_name, prediction in zip(file_names,clip):
        f_name = f_name.replace(".png","")
        f.write(f"{f_name} {prediction}\n")